In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
# read in Own2InvHldTO data
file_name = '/content/gdrive/My Drive/capstone/Own2InvHldTO.csv'
inv_df = pd.DataFrame(pd.read_csv(file_name))

In [5]:
inv_df.head()

,LgcyInvestorId,IncWrntsCert,CalcBasisCode,EffectFromDate,EffectToDate,InvestorId,OwnTurnover,OwnTurnoverRtng
0,9943268,0,1,2018-07-01 00:00:00,2018-07-31 00:00:00,5.061165e+09,39.0603,LOW
1,9943268,0,1,2018-06-01 00:00:00,2018-06-30 00:00:00,5.061165e+09,39.0603,LOW
2,9943268,1,2,2020-04-01 00:00:00,2020-06-30 00:00:00,5.061165e+09,93.0138,MED
3,9943268,1,2,2020-01-01 00:00:00,2020-03-31 00:00:00,5.061165e+09,91.5372,MED
4,9943268,1,2,2019-10-01 00:00:00,2019-12-31 00:00:00,5.061165e+09,99.4172,HIGH


In [6]:
inv_df.EffectToDate = pd.to_datetime(inv_df.EffectToDate)
inv_df.EffectFromDate = pd.to_datetime(inv_df.EffectFromDate)

In [7]:
# read in Own2InvAstAlloc data
file_name1 = '/content/gdrive/My Drive/capstone/Own2InvAstAlloc.csv'
allo_df = pd.DataFrame(pd.read_csv(file_name1))

In [8]:
allo_df.head()

,LgcyInvestorId,FullName,InvestorId,AstAllocReg,EqAstAlloc,FIAstAlloc,CashAstAlloc,TotAstAlloc
0,400286,"Chilton Investment Company, LLC",5.000008e+09,AAREG7,AARANGE8,AARANGE1,AARANGE1,AARANGE1
1,400614,"Greenway Partners, L.P._NLE",4.295975e+09,AAREG7,AARANGE1,AARANGE1,AARANGE1,AARANGE1
2,403412,"Osterweis Capital Management, LLC",5.000059e+09,AAREG4,AARANGE3,AARANGE2,AARANGE2,AARANGE3
3,405602,GE Asset Management Inc._NLE,5.000035e+09,AAREG3,AARANGE9,AARANGE6,AARANGE2,AARANGE9
4,413868,"Wells Fargo Bank of South Dakota, N.A._NLE",NaN,AAREG7,AARANGE5,AARANGE4,AARANGE2,AARANGE5


In [9]:
allo_name_df = allo_df[['LgcyInvestorId','FullName']]
allo_name_df.head()

,LgcyInvestorId,FullName
0,400286,"Chilton Investment Company, LLC"
1,400614,"Greenway Partners, L.P._NLE"
2,403412,"Osterweis Capital Management, LLC"
3,405602,GE Asset Management Inc._NLE
4,413868,"Wells Fargo Bank of South Dakota, N.A._NLE"


In [10]:
# read in list of investors of interest
inv_list = pd.read_csv('/content/gdrive/My Drive/capstone/ListOfInvestor.csv')
inv_ls_df = pd.DataFrame(inv_list)
investor_ls = inv_list.LgcyInvestorID.to_list()

In [11]:
inv_ls_df.head()

,LgcyInvestorID
0,5436717
1,2003762
2,2004523
3,2003710
4,2002451


In [12]:
# select all investors in the list that count their turnover in quarter
inv_list_df = inv_df[inv_df.LgcyInvestorId.isin(investor_ls)]
inv_qu_list_df = inv_list_df[(inv_list_df.CalcBasisCode == 2) & (inv_list_df.IncWrntsCert == 1)]
inv_qu_list_df

,LgcyInvestorId,IncWrntsCert,CalcBasisCode,EffectFromDate,EffectToDate,InvestorId,OwnTurnover,OwnTurnoverRtng
629,7016524,1,2,2020-04-01,2020-06-30,5.038078e+09,20.8623,LOW
630,7016524,1,2,2020-01-01,2020-03-31,5.038078e+09,7.4348,LOW
631,7016524,1,2,2019-10-01,2019-12-31,5.038078e+09,8.5476,LOW
632,7016524,1,2,2019-07-01,2019-09-30,5.038078e+09,18.6006,LOW
633,7016524,1,2,2019-04-01,2019-06-30,5.038078e+09,40.8454,LOW
...,...,...,...,...,...,...,...,...
650119,7185253,1,2,2019-04-01,2019-06-30,5.000757e+09,64.9708,MED
650120,7185253,1,2,2019-07-01,2019-09-30,5.000757e+09,66.7172,MED
650121,7185253,1,2,2019-10-01,2019-12-31,5.000757e+09,66.2099,MED
650122,7185253,1,2,2020-01-01,2020-03-31,5.000757e+09,66.1567,MED


In [13]:
inv_qu_list_df_r = inv_qu_list_df[['LgcyInvestorId','EffectToDate','OwnTurnover']]
inv_qu_list_df_r.head(20)

,LgcyInvestorId,EffectToDate,OwnTurnover
629,7016524,2020-06-30,20.8623
630,7016524,2020-03-31,7.4348
631,7016524,2019-12-31,8.5476
632,7016524,2019-09-30,18.6006
633,7016524,2019-06-30,40.8454
634,7016524,2019-03-31,59.3847
635,7016524,2018-12-31,67.8173
636,7016524,2018-09-30,53.8877
637,7016524,2018-06-30,38.1639
638,7016524,2018-03-31,20.7433


In [14]:
inv_qu_list_df_r.to_csv(path_or_buf = '/content/gdrive/My Drive/capstone/inv_qu_list_df.csv')

**1. Fill in missing value**

First we need to know how many records for each investor.  Count the number of records for each investor, if find any investor has records number lower than the correct one, fill in 0.

In [41]:
file_name_df = '/content/gdrive/My Drive/capstone/df.csv'
df = pd.DataFrame(pd.read_csv(file_name_df))

In [42]:
df.head()

,LgcyInvestorId,EffectToDate,OwnTurnover
0,2000264,2016-09-30,20.8623
1,2000264,2016-12-31,7.4348
2,2000264,2017-03-31,8.5476
3,2000264,2017-06-30,18.6006
4,2000264,2017-09-30,40.8454


In [43]:
#find all investors in the list
all_inv = df.LgcyInvestorId.drop_duplicates().to_list()
len(all_inv)

228

In [44]:
def findInvNum(all_inv,df_list):
  num_ls = []
  for elem in all_inv:
    count = 0
    for num in df_list:
      if elem == num:
        count += 1
    num_ls.append(count)
  return num_ls

In [45]:
df_list = df.LgcyInvestorId.to_list()
num_ls = findInvNum(all_inv,df_list)

In [46]:
c = {
    "LgcyInvestorId":all_inv,
    "count":num_ls
}
data_check=pd.DataFrame(c).sort_values(by=['LgcyInvestorId'])
data_check.head()

,LgcyInvestorId,count
0,2000264,16
1,2001934,16
2,2001935,16
3,2001989,16
4,2001999,16


In [47]:
#check if all the investors filled the zero as missing value
#the expected result is nothing shows in the below table
data_check.loc[data_check['count'] != 16]

,LgcyInvestorId,count


**2. Find the Euclidean distance**

Find the method to compute the Euclidean distance between two time series data.

In [57]:
# make the time-series table to matrix
def dfMatrix(all_inv, df):
  matrix = []
  for elem in all_inv:
    time_series = []
    for i in range(len(df)):
      if elem == df['LgcyInvestorId'][i]:
        time_series.append(df['OwnTurnover'][i])
    matrix.append(time_series)
  return matrix

In [59]:
df_matrix = dfMatrix(all_inv,df)
df_matrix[:5]

[[20.8623,
  7.4348,
  8.5476,
  18.6006,
  40.8454,
  59.3847,
  67.8173,
  53.8877,
  38.1639,
  20.7433,
  145.2031,
  142.91,
  211.68,
  17.23,
  0.0,
  0.0],
 [11.8181,
  10.1652,
  10.4574,
  12.2989,
  12.7081,
  11.7884,
  14.4976,
  13.0898,
  15.7996,
  16.4254,
  18.1095,
  20.7776,
  20.167,
  19.8767,
  19.3727,
  17.3699],
 [41.1877,
  38.0313,
  39.4283,
  35.6277,
  39.4725,
  42.114,
  43.3943,
  48.6795,
  48.4185,
  50.5321,
  46.641000000000005,
  49.4782,
  49.7565,
  49.8499,
  47.4245,
  58.1764],
 [67.3927,
  57.5475,
  59.425,
  61.1245,
  62.1462,
  64.226,
  62.9307,
  63.1852,
  64.6962,
  62.4586,
  66.0049,
  65.4712,
  64.6412,
  65.2642,
  63.3851,
  64.9578],
 [19.1317,
  15.3662,
  13.0782,
  14.2334,
  15.0065,
  15.9249,
  15.924000000000001,
  15.9152,
  17.0455,
  18.7073,
  18.1425,
  19.0751,
  18.4135,
  17.6517,
  16.9085,
  16.8328]]

**3. Find the turnover matrix**

Find the turnover distance matrix.

In [61]:
from scipy.spatial.distance import pdist, squareform
matrix = squareform(pdist(df_matrix))

In [62]:
matrix

array([[   0.        ,  276.85076749,  236.36825158,  237.77882358,
         276.85320694,  266.81143259,  297.3884141 ,  236.2388272 ,
         287.57809664,  566.98611361,  238.02324712,  252.49249776,
         237.97772592,  212.37512413,  255.08876797,  795.87434326,
         281.99823971,  477.41845526,  258.83766097,  607.48715838,
         244.34145713,  256.78693912,  454.03175813,  493.17817019,
         428.61055879,  363.22048692,  250.02505477,  285.31059192,
         264.18900657,  350.47080694,  319.983937  ,  209.78602001,
         180.0894496 ,  214.22820435,  251.52885436,  242.41722787,
         235.77840716,  399.23741334,  260.36145766,  203.49925685,
         257.01277968,  251.81605094,  238.31466994,  288.63915794,
         275.36741813,  217.06006499,  297.39506079,  250.54966811,
         317.92377796,  348.67784453,  243.21094792,  322.75621617,
         445.99748378,  253.99067747,  326.25059505,  358.09756893,
         402.51115146,  653.07373246,  214.90604

In [65]:
matrix_df = pd.DataFrame(matrix)
matrix_df.index = all_inv
matrix_df.columns = all_inv
matrix_df.head()

,2000264,2001934,2001935,2001989,2001999,2002019,2002142,2002146,2002158,2002179,2002180,2002195,2002205,2002222,2002292,2002295,2002303,2002362,2002389,2002446,2002451,2002468,2002496,2002506,2002511,2002548,2002582,2002628,2002658,2002659,2002678,2002695,2002713,2002715,2002801,2002819,2002823,2002848,2002867,2002875,2002921,2002946,2002947,2002962,2002964,2002975,2003003,2003020,2003028,2003064,2003101,2003121,2003131,2003135,2003144,2003147,2003150,2003171,2003177,2003228,2003230,2003245,2003251,2003263,2003294,2003299,2003306,2003332,2003354,2003356,2003361,2003403,2003543,2003597,2003671,2003675,2003700,2003710,2003716,2003762,2003773,2003800,2003819,2003828,2003838,2004000,2004134,2004154,2004260,2004282,2004318,2004344,2004351,2004490,2004523,2004601,2004609,2004627,2004684,2004887,2004904,2004971,2005078,2005124,2005645,2005801,2005821,2005881,2005893,2005964,2006020,2006033,2006150,2006207,2006241,2006332,2006430,2006452,2006474,2006588,2006672,2006800,2006825,2006840,2006922,2006955,2006990,2007039,2007217,2007403,2007567,2007683,2113769,2247678,2275507,2380201,2386352,2401336,2401343,2412347,2412732,2414807,2416337,2420555,2423945,2426047,2447262,2449877,2452904,2455426,2463029,2487483,2490223,2490230,2494658,2498287,2500087,2500944,2502059,2502377,2503132,2507775,2529162,2546136,2575181,2575182,2578454,2587535,2592292,2620417,2625886,2627998,2639138,2646754,2655110,2785553,2906273,2926442,2930530,2931843,2994959,3015075,3318171,3321607,3365036,3676543,3706224,3828352,3965008,4040011,4059340,4889409,4924814,4950170,4962258,5050280,5200477,5205742,5206490,5206923,5275446,5308615,5436717,5480445,5505145,5520904,5529187,5713469,5819294,5844860,5915225,5986931,6053249,6129655,6139212,6248967,6271047,6921752,7016524,7185253,9098246,9113038,9969058,10061989,10075382,10110259,10233291,10235680
2000264,0.000000,276.850767,236.368252,237.778824,276.853207,266.811433,297.388414,236.238827,287.578097,566.986114,238.023247,252.492498,237.977726,212.375124,255.088768,795.874343,281.998240,477.418455,258.837661,607.487158,244.341457,256.786939,454.031758,493.178170,428.610559,363.220487,250.025055,285.310592,264.189007,350.470807,319.983937,209.786020,180.089450,214.228204,251.528854,242.417228,235.778407,399.237413,260.361458,203.499257,257.012780,251.816051,238.314670,288.639158,275.367418,217.060065,297.395061,250.549668,317.923778,348.677845,243.210948,322.756216,445.997484,253.990677,326.250595,358.097569,402.511151,653.073732,214.906049,313.560945,318.657628,467.237924,462.817313,242.955760,322.152371,489.008349,242.624790,281.290971,270.675056,325.641165,484.921628,256.054780,195.235999,272.687676,249.281490,232.885979,483.662475,215.326157,296.304897,243.501911,302.916323,324.044022,401.080676,529.821807,1125.975287,268.982337,343.676487,284.358870,241.825744,300.858546,323.012999,288.564253,292.390871,243.909933,256.341672,328.012711,262.982081,250.295304,378.037580,338.071129,287.058587,247.325258,256.331115,269.347554,308.979242,376.253234,307.460970,258.329823,297.658626,303.979423,219.028348,292.062856,367.431014,223.541641,356.571119,343.309007,270.220423,278.335275,329.218305,217.780168,287.874005,246.520301,221.693659,257.916046,273.008417,327.385518,797.970608,553.288612,769.743324,434.560276,301.665734,310.679949,516.910269,306.908747,485.618319,244.261647,285.848807,241.482097,274.662553,540.946258,248.414540,287.048843,285.192434,274.508816,616.626220,274.951898,292.162136,467.818155,666.908228,613.170676,477.973271,228.053868,316.577018,333.603673,256.210217,222.447336,364.552954,246.899553,260.836257,249.041726,533.612299,615.089697,376.130588,269.110113,246.351007,244.694310,242.112439,269.629581,271.210644,239.004596,233.694088,271.816301,251.664399,244.006827,233.813875,256.502782,243.316494,297.239367,237.786789,226.027773,242.870979,315.867577,238.781795,850.797936,546.770854,305.632221,297.790359,361.280010,266.392517,485.846560,741.979491,288.232467,252.457054,421.024069,327.732038,460.336872,258.066

In [66]:
matrix_df.to_csv(path_or_buf = '/content/gdrive/My Drive/capstone/turnover_matrix.csv')